
# Quantum Networking — Student BB84 Notebook (Final)
This notebook defines the student-side BB84 implementation and runs the end-to-end simulation.
It is wired to work with the finalized `InteractiveQuantumHost` and `QuantumAdapter`:
- Provides **realistic measurement** (random outcomes on basis mismatch)
- Implements the required **student API** that the host calls
- Exposes **global** `alice` and `bob` and also **registers** them if a registry is present
- Exports a **plugin** for fallback loading


In [ ]:

import sys, pathlib, random
project_root = pathlib.Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
print("PYTHONPATH ok ->", project_root)


In [ ]:

def prepare_quantum_state(bit, basis):
    """Return symbolic BB84 states understood by InteractiveQuantumHost when QuTiP is absent."""
    if basis == 0 or basis == 'Z':
        return '|0⟩' if bit == 0 else '|1⟩'
    else:  # X basis
        return '|+⟩' if bit == 0 else '|-⟩'

def measure_quantum_state(quantum_state, measurement_basis):
    """Physically correct measurement for BB84 symbolic states.
    On basis mismatch, outcomes are random (50/50).
    measurement_basis: 0=Z, 1=X (or 'Z'/'X')
    """
    m = 0 if measurement_basis in (0, 'Z') else 1
    if m == 0:  # Z-basis
        if quantum_state in ['|0⟩', '|1⟩', '|0>', '|1>']:
            return 0 if quantum_state in ['|0⟩', '|0>'] else 1
        return random.randint(0, 1)
    else:  # X-basis
        if quantum_state in ['|+⟩', '|-⟩', '|+>', '|->']:
            return 0 if quantum_state in ['|+⟩', '|+>'] else 1
        return random.randint(0, 1)


In [ ]:

class StudentQuantumHost:
    """Student-side BB84 implementation that plugs into InteractiveQuantumHost.
    The bridge sets 'self.host' to the actual host instance.
    Required methods:
      - bb84_send_qubits(num_qubits)
      - process_received_qbit(qbit, from_channel)
      - bb84_reconcile_bases(their_bases)
      - bb84_estimate_error_rate(their_bits_sample)
    """

    def __init__(self, name):
        self.name = name
        self.host = None  # set by InteractiveQuantumHost.attach_student / set_host()

    def set_host(self, host):
        self.host = host

    def _rand_basis(self):
        return random.choice(['Z', 'X'])

    def bb84_send_qubits(self, num_qubits: int):
        if self.host is None:
            raise RuntimeError(f"{self.name}: host not attached")

        # Reset protocol state on the host (if available)
        if hasattr(self.host, 'reset_qkd_state'):
            self.host.reset_qkd_state()

        bits = [random.randint(0, 1) for _ in range(num_qubits)]
        bases = [self._rand_basis() for _ in range(num_qubits)]
        self.host.basis_choices = bases

        chan = self.host.get_channel()
        if chan is None:
            print(f"❌ {self.name}: No quantum channel available")
            return False

        for b, bas in zip(bits, bases):
            q = self.host.prepare_qubit(basis=bas, bit=b)
            self.host.send_qubit(q, chan)

        print(f"🚀 {self.name}: Sent {num_qubits} BB84 states")
        try:
            self.host.learning_stats['qubits_sent'] += num_qubits
        except Exception:
            pass
        return True

    def process_received_qbit(self, qbit, from_channel):
        if self.host is None:
            raise RuntimeError(f"{self.name}: host not attached")

        bas = self._rand_basis()
        bit = self.host.measure_qubit(qbit, bas)
        self.host.measurement_outcomes.append(bit)
        try:
            self.host.learning_stats['qubits_received'] += 1
        except Exception:
            pass
        return True

    def bb84_reconcile_bases(self, their_bases):
        if self.host is None:
            raise RuntimeError(f"{self.name}: host not attached")

        ours = self.host.basis_choices
        shared = [i for i, (a, b) in enumerate(zip(ours, their_bases)) if a == b]
        self.host.shared_bases_indices = shared
        # Notify peer
        self.host.send_classical_data({'type': 'shared_bases_indices', 'data': shared})
        print(f"🤝 {self.name}: Shared indices = {len(shared)}")
        return shared

    def bb84_estimate_error_rate(self, their_bits_sample):
        if self.host is None:
            raise RuntimeError(f"{self.name}: host not attached")

        errors = 0
        comps = 0
        outcomes = self.host.measurement_outcomes
        for bit, idx in their_bits_sample:
            if 0 <= idx < len(outcomes):
                comps += 1
                if outcomes[idx] != bit:
                    errors += 1

        qber = (errors / comps) if comps else 0.0
        try:
            self.host.learning_stats['error_rates'].append(qber)
        except Exception:
            pass

        print(f"📈 {self.name}: QBER = {qber:.2%} on {comps} samples")
        # Signal completion
        self.host.send_classical_data({'type': 'complete'})
        return qber


In [ ]:

# Create global student instances
alice = StudentQuantumHost("Alice")
bob   = StudentQuantumHost("Bob")

# Expose in builtins for any legacy/global-based bridges
import builtins
builtins.alice = alice
builtins.bob = bob

print("Created StudentQuantumHost instances -> alice, bob (globals set)" )

# Try optional registry registration
try:
    from quantum_network.student_registry import REGISTRY
    REGISTRY.register("Alice", alice)
    REGISTRY.register("Bob", bob)
    print("Registered alice/bob in REGISTRY")
except Exception as e:
    print("Registry not available (that's OK):", e)

# Export a plugin module so InteractiveQuantumHost can load it if needed
plugin_code = r'''
class StudentImplementation:
    """Plugin form expected by InteractiveQuantumHost._load_student_plugin_from_file.
    It is constructed as StudentImplementation(host).
    Implements the required methods directly using the provided host.
    """
    def __init__(self, host):
        self.host = host
        self.name = getattr(host, 'name', 'Student')

    def _rand_basis(self):
        import random
        return random.choice(['Z', 'X'])

    def bb84_send_qubits(self, num_qubits: int):
        host = self.host
        if hasattr(host, 'reset_qkd_state'):
            host.reset_qkd_state()
        import random
        bits = [random.randint(0, 1) for _ in range(num_qubits)]
        bases = [self._rand_basis() for _ in range(num_qubits)]
        host.basis_choices = bases
        chan = host.get_channel()
        if chan is None:
            print(f"❌ {self.name}: No quantum channel available")
            return False
        for b, bas in zip(bits, bases):
            q = host.prepare_qubit(basis=bas, bit=b)
            host.send_qubit(q, chan)
        print(f"🚀 {self.name}: Sent {num_qubits} BB84 states (plugin)")
        return True

    def process_received_qbit(self, qbit, from_channel):
        host = self.host
        bit = host.measure_qubit(qbit, self._rand_basis())
        host.measurement_outcomes.append(bit)
        return True

    def bb84_reconcile_bases(self, their_bases):
        host = self.host
        ours = host.basis_choices
        shared = [i for i, (a, b) in enumerate(zip(ours, their_bases)) if a == b]
        host.shared_bases_indices = shared
        host.send_classical_data({'type': 'shared_bases_indices', 'data': shared})
        return shared

    def bb84_estimate_error_rate(self, their_bits_sample):
        host = self.host
        errors = 0
        comps = 0
        outcomes = host.measurement_outcomes
        for bit, idx in their_bits_sample:
            if 0 <= idx < len(outcomes):
                comps += 1
                if outcomes[idx] != bit:
                    errors += 1
        qber = (errors / comps) if comps else 0.0
        host.send_classical_data({'type': 'complete'})
        return qber
'''
plugin_path = pathlib.Path('student_plugin.py')
plugin_path.write_text(plugin_code, encoding='utf-8')

status = {
    "student_implementation_ready": True,
    "student_plugin_module": "student_plugin",
    "student_plugin_class": "StudentImplementation"
}
path_status = pathlib.Path('student_implementation_status.json')
path_status.write_text(__import__('json').dumps(status, indent=2), encoding='utf-8')

print("Exported plugin -> student_plugin.py and student_implementation_status.json")


In [ ]:

# Quick sanity check of helper functions:
print('Example Z-prep bit=1:', prepare_quantum_state(1, 'Z'))
print('Measure |+> in Z (random):', [measure_quantum_state('|+⟩', 0) for _ in range(5)])


In [ ]:

# Import the fixed runner that accepts (alice, bob) instances
from complete_quantum_simulation_fixed import run_complete_quantum_simulation_with_instances

print("Starting complete quantum simulation (using student instances)...")
success = run_complete_quantum_simulation_with_instances(alice, bob)
print("✅ Success" if success else "❌ Failed")

# Optional sanity: verify that our instances were attached to hosts by the simulation
try:
    # If your runner attaches host refs back on the student impls
    print("alice.host present:", hasattr(alice, 'host') and alice.host is not None)
    print("bob.host present:", hasattr(bob, 'host') and bob.host is not None)
except Exception as e:
    print("Post-run check skipped:", e)
